In [11]:
import nltk
# solamente una vez
nltk.download('treebank')

[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\Sergio\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!


True

In [12]:
from nltk.corpus import treebank

corpus = treebank.sents()

In [13]:
corpus

[['Pierre', 'Vinken', ',', '61', 'years', 'old', ',', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'Nov.', '29', '.'], ['Mr.', 'Vinken', 'is', 'chairman', 'of', 'Elsevier', 'N.V.', ',', 'the', 'Dutch', 'publishing', 'group', '.'], ...]

In [14]:
from nltk import ngrams # bi-grama

In [15]:
oraciones_bigramas = [list(ngrams(oracion,2)) for oracion in corpus]

In [16]:
bigramas = [list(bigrama) for oracion in oraciones_bigramas for bigrama in oracion]

In [17]:
bigramas

[['Pierre', 'Vinken'],
 ['Vinken', ','],
 [',', '61'],
 ['61', 'years'],
 ['years', 'old'],
 ['old', ','],
 [',', 'will'],
 ['will', 'join'],
 ['join', 'the'],
 ['the', 'board'],
 ['board', 'as'],
 ['as', 'a'],
 ['a', 'nonexecutive'],
 ['nonexecutive', 'director'],
 ['director', 'Nov.'],
 ['Nov.', '29'],
 ['29', '.'],
 ['Mr.', 'Vinken'],
 ['Vinken', 'is'],
 ['is', 'chairman'],
 ['chairman', 'of'],
 ['of', 'Elsevier'],
 ['Elsevier', 'N.V.'],
 ['N.V.', ','],
 [',', 'the'],
 ['the', 'Dutch'],
 ['Dutch', 'publishing'],
 ['publishing', 'group'],
 ['group', '.'],
 ['Rudolph', 'Agnew'],
 ['Agnew', ','],
 [',', '55'],
 ['55', 'years'],
 ['years', 'old'],
 ['old', 'and'],
 ['and', 'former'],
 ['former', 'chairman'],
 ['chairman', 'of'],
 ['of', 'Consolidated'],
 ['Consolidated', 'Gold'],
 ['Gold', 'Fields'],
 ['Fields', 'PLC'],
 ['PLC', ','],
 [',', 'was'],
 ['was', 'named'],
 ['named', '*-1'],
 ['*-1', 'a'],
 ['a', 'nonexecutive'],
 ['nonexecutive', 'director'],
 ['director', 'of'],
 ['of', 't

In [18]:
num_palabras = 5000

In [19]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizador = Tokenizer(num_words=num_palabras,oov_token="UNK")

In [20]:
tokenizador.fit_on_texts(bigramas)

In [21]:
len_vocabulario = len(tokenizador.word_index)+1
len_vocabulario

11388

In [22]:
secuencias = tokenizador.texts_to_sequences(bigramas)
secuencias

[[1, 3731],
 [3731, 2],
 [2, 2014],
 [2014, 85],
 [85, 334],
 [334, 2],
 [2, 45],
 [45, 2390],
 [2390, 3],
 [3, 126],
 [126, 28],
 [28, 7],
 [7, 2015],
 [2015, 327],
 [327, 452],
 [452, 2016],
 [2016, 6],
 [43, 3731],
 [3731, 19],
 [19, 177],
 [177, 4],
 [4, 1],
 [1, 2905],
 [2905, 2],
 [2, 3],
 [3, 2906],
 [2906, 670],
 [670, 141],
 [141, 6],
 [3682, 1],
 [1, 2],
 [2, 1129],
 [1129, 85],
 [85, 334],
 [334, 9],
 [9, 600],
 [600, 177],
 [177, 4],
 [4, 1],
 [1, 1037],
 [1037, 884],
 [884, 885],
 [885, 2],
 [2, 33],
 [33, 494],
 [494, 10],
 [10, 7],
 [7, 2015],
 [2015, 327],
 [327, 4],
 [4, 55],
 [55, 1038],
 [1038, 430],
 [430, 2907],
 [2907, 6],
 [7, 635],
 [635, 4],
 [4, 1039],
 [1039, 708],
 [708, 319],
 [319, 12],
 [12, 12],
 [12, 5],
 [5, 123],
 [123, 1366],
 [1366, 2391],
 [2391, 1539],
 [1539, 37],
 [37, 886],
 [886, 7],
 [7, 192],
 [192, 1040],
 [1040, 4],
 [4, 1236],
 [1236, 1751],
 [1751, 275],
 [275, 7],
 [7, 141],
 [141, 4],
 [4, 535],
 [535, 3732],
 [3732, 12],
 [12, 5],
 [5

In [23]:
len(secuencias)

96762

In [24]:
import tensorflow as tf

dataset = tf.data.Dataset.from_tensor_slices(secuencias)
batch_size = 100 # Hiperparámetro
dataset = dataset.shuffle(batch_size).batch(batch_size)

Se define la RNN 

In [25]:
b = tf.Variable(tf.random.normal([num_palabras]))
W = tf.Variable(tf.random.normal([200,num_palabras]))

E = tf.Variable(tf.random.normal([num_palabras,200]))

optimizador = tf.keras.optimizers.experimental.SGD(learning_rate=0.1)

epochs = 10

for _ in range(epochs):
  loss_epoch = 0
  for bigramas in dataset:
    x = bigramas[:,0]
    y = bigramas[:,1]
    with tf.GradientTape() as gt:
      # Capa 1: Embeddings: transformación de palabra a vector
      embedding = tf.nn.embedding_lookup(E,x)

      # Capa 2 (salida)
      operacion_matricial = tf.matmul(embedding,W)+b

      x_ent = tf.nn.sparse_softmax_cross_entropy_with_logits(
          logits = operacion_matricial,
          labels = y
      )

      loss = tf.reduce_mean(x_ent)
      optimizador.minimize(loss,[W,b,E],gt) # Poderosa

    loss_epoch += loss/len(dataset)
  print(loss_epoch)

tf.Tensor(37.0697, shape=(), dtype=float32)
tf.Tensor(28.778715, shape=(), dtype=float32)
tf.Tensor(25.226503, shape=(), dtype=float32)
tf.Tensor(22.91881, shape=(), dtype=float32)
tf.Tensor(21.190308, shape=(), dtype=float32)
tf.Tensor(19.823957, shape=(), dtype=float32)
tf.Tensor(18.700747, shape=(), dtype=float32)
tf.Tensor(17.763292, shape=(), dtype=float32)
tf.Tensor(16.963734, shape=(), dtype=float32)
tf.Tensor(16.268776, shape=(), dtype=float32)


Ahora si para buscar una palabra se busca el num ID  despupes los vectores más cercanos con las distancias coseno 

In [26]:
palabra_entrada = ["bank"]
num_id = tokenizador.texts_to_sequences(palabra_entrada)
embedding = tf.nn.embedding_lookup(E,num_id)

distancias_coseno = tf.matmul(E, tf.reshape(embedding,(-1,1)))
distancias_coseno = tf.squeeze(distancias_coseno)

import numpy as np

top_k = 6
top_indices = np.argsort(-distancias_coseno)[:top_k]

tokenizador.sequences_to_texts([top_indices])

['bank editorial stop abandoned creation event']